In [1]:
import pandas as pd
def _load_vaccination_per_model_data_df():
        vaccination_df = pd.read_csv("https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations-by-manufacturer.csv?raw=true")
        return vaccination_df
vac_per_manu_factor = _load_vaccination_per_model_data_df()

In [2]:
print("Columnas: ", vac_per_manu_factor.columns)
print("Filas: ", vac_per_manu_factor.shape[0])
print("Paises: ", vac_per_manu_factor.location.unique().shape[0])
print("Rango de las fechas: ", vac_per_manu_factor.date.min(), " - ", vac_per_manu_factor.date.max())
print("Fabricantes: ", vac_per_manu_factor.vaccine.unique())
print("Lista de paises: ", vac_per_manu_factor.location.unique())

Columnas:  Index(['location', 'date', 'vaccine', 'total_vaccinations'], dtype='object')
Filas:  59111
Paises:  44
Rango de las fechas:  2020-12-04  -  2023-04-19
Fabricantes:  ['Oxford/AstraZeneca' 'Sinopharm/Beijing' 'Sputnik V' 'Pfizer/BioNTech'
 'Moderna' 'CanSino' 'Sputnik Light' 'Johnson&Johnson' 'Novavax'
 'Sanofi/GSK' 'Valneva' 'Medicago' 'Sinovac' 'Covaxin' 'SKYCovione']
Lista de paises:  ['Argentina' 'Austria' 'Belgium' 'Bulgaria' 'Canada' 'Chile' 'Croatia'
 'Cyprus' 'Czechia' 'Denmark' 'Ecuador' 'Estonia' 'Finland' 'France'
 'Germany' 'Hong Kong' 'Hungary' 'Iceland' 'Ireland' 'Italy' 'Japan'
 'Latvia' 'Liechtenstein' 'Lithuania' 'Luxembourg' 'Malta' 'Nepal'
 'Netherlands' 'Norway' 'Peru' 'Poland' 'Portugal' 'Romania' 'Slovakia'
 'Slovenia' 'South Africa' 'South Korea' 'Spain' 'Sweden' 'Switzerland'
 'Ukraine' 'United States' 'Uruguay' 'European Union']


In [9]:
# Lista con todos los paises
paises = vac_per_manu_factor.location.unique()
# Quitamos chile de la lista
paises  = paises[ paises != "Chile"]
# Quitamos Hong Kong de la lista
paises  = paises[ paises != "Hong Kong"]
# Qitamos Iceland de la lista
paises  = paises[ paises != "Iceland"]
# Quitamos Japan de la lista
paises  = paises[ paises != "Japan"]
# Quitamos Nepal de la lista
paises  = paises[ paises != "Nepal"]
# Quitamos Peru de la lista
paises  = paises[ paises != "Peru"]
# Quitamos Romania de la lista
paises  = paises[ paises != "Romania"]
# Quitamos South Africa de la lista
paises  = paises[ paises != "South Africa"]
# Quitamos South Korea de la lista
paises  = paises[ paises != "South Korea"]
# Quitamos Ukraine de la lista
paises  = paises[ paises != "Ukraine"]
# Quitamos Uruguay de la lista
paises  = paises[ paises != "Uruguay"]
for pais in paises:
    print("Procesando: ", pais)
    vac_per_manu_factor_sp = vac_per_manu_factor[vac_per_manu_factor.location == pais]
    # Vamos a quedarnos con el mes de enero
    vac_per_manu_factor_sp_jan = vac_per_manu_factor_sp[vac_per_manu_factor_sp.date.str.contains("2021-01")]
    ############################################
    # Vamos a añadir filas de forma que no tengamos saltos de fechas
    # Pero las vamos a duplicar 4 veces
    # De forma que el 2021-01-01 se repita 4 veces
    dates = pd.DataFrame(pd.date_range(start='2021-01-01', end='2021-12-31'), columns=['date'])
    dates['date'] = dates['date'].dt.strftime('%Y-%m-%d')
    # Ahora repetimos 4 veces las fechas por el número de paises por el número de fabricantes
    print(len(vac_per_manu_factor_sp_jan.vaccine.unique()) -1)
    dates = pd.concat([dates]* (len(vac_per_manu_factor_sp_jan.vaccine.unique()) -1))
    # Ordenamos por fecha
    dates = dates.sort_values(by=['date'])
    #vac_per_manu_factor_sp_jan_all = pd.merge(dates,vac_per_manu_factor_sp_jan, on='date', how='inner')
    vacunas = vac_per_manu_factor_sp_jan.vaccine.unique()
    print(vacunas)
    # Elimniamos el primer elemento que es 0
    vacunas = vacunas[1:]
    # repetimos 4 veces las vacunas
    vacunas = pd.concat([pd.Series(vacunas)]*365).tolist()
    #print(vacunas)
    # Ahora vamos a sacar una lista con todos los paises
    countries = vac_per_manu_factor_sp_jan.location.unique()
    # Añadimos las columnas de location vaccine total_vaccinations al dataframe
    dates['location'] = pais
    # En vaccine vamos a repetir 4 veces el nombre de la vacuna
    dates['vaccine'] = vacunas
    dates['total_vaccinations'] = 0
    # Resetamos el indice
    dates = dates.reset_index(drop=True)
    ############################################
    # Vamos a recorrer el dataframe que contiene el número de vacunas por fabricante
    # y vamos a ir completandolo en el dataframe de dates
    data = vac_per_manu_factor_sp.reset_index(drop=True)
    vacunas = vac_per_manu_factor_sp_jan.vaccine.unique()
    vacunas_dict = {}
    for vacuna in vacunas:
        vacunas_dict[vacuna] = data[data.vaccine == vacuna].reset_index(drop=True)
    #print(vacunas_dict)
    ############################################
    # Recorremos el diccionario
    for vacuna in vacunas_dict:
        
        # mientras de expeción, incrementamos el contador
        try:
            LDWD = dates[dates.vaccine == vacuna].iloc[0].date
        except:
            LDWD = "2021-01-01"
        
        cum_vaccinations = 0
        for i in range(len(vacunas_dict[vacuna])):
            # COgemos la primera fila
            fila = vacunas_dict[vacuna].iloc[i]
            current_date = fila.date
            cum_vaccinations = fila.total_vaccinations - cum_vaccinations
            if i == 0:
                diff_dates = (pd.to_datetime(current_date) - pd.to_datetime(LDWD)).days + 1
            else:
                diff_dates = (pd.to_datetime(current_date) - pd.to_datetime(LDWD)).days
            #print(diff_dates)
            total_vaccinations = cum_vaccinations // diff_dates
            #print(fila.total_vaccinations)
            #print(current_date)
            #print(total_vaccinations)
            cum_vaccinations = fila.total_vaccinations
            dates.loc[(dates.vaccine == vacuna) & (dates.date <= current_date) & (dates.date >= LDWD), 'total_vaccinations'] = total_vaccinations
            LDWD = current_date
    # Store the dataframe in a csv file
    pd.DataFrame(dates).to_csv("vacunas/vacunas_por_fabricante_" + pais + ".csv", index=False)


Procesando:  Argentina
5
['Oxford/AstraZeneca' 'Sputnik V' 'Pfizer/BioNTech' 'Sinopharm/Beijing'
 'Moderna' 'CanSino']
Procesando:  Austria
6
['Johnson&Johnson' 'Moderna' 'Novavax' 'Oxford/AstraZeneca'
 'Pfizer/BioNTech' 'Sanofi/GSK' 'Valneva']
Procesando:  Belgium
4
['Johnson&Johnson' 'Moderna' 'Novavax' 'Oxford/AstraZeneca'
 'Pfizer/BioNTech']
Procesando:  Bulgaria
1
['Pfizer/BioNTech' 'Moderna']
Procesando:  Canada
1
['Moderna' 'Pfizer/BioNTech']
Procesando:  Croatia
2
['Pfizer/BioNTech' 'Moderna' 'Oxford/AstraZeneca']
Procesando:  Cyprus
1
['Pfizer/BioNTech' 'Moderna']
Procesando:  Czechia
4
['Moderna' 'Pfizer/BioNTech' 'Oxford/AstraZeneca' 'Covaxin' 'Sinovac']
Procesando:  Denmark
3
['Moderna' 'Pfizer/BioNTech' 'Johnson&Johnson' 'Oxford/AstraZeneca']
Procesando:  Ecuador
1
['Pfizer/BioNTech' 'Sinovac']
Procesando:  Estonia
1
['Pfizer/BioNTech' 'Moderna']
Procesando:  Finland
5
['Johnson&Johnson' 'Moderna' 'Novavax' 'Oxford/AstraZeneca'
 'Pfizer/BioNTech' 'Valneva']
Procesando:  Fr

In [10]:
# Vamos a leer los csvs de un directorio y los vamos a unir en un solo dataframe
path = 'vacunas'
import glob
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
vacunas_por_fabricante = pd.concat(li, axis=0, ignore_index=True)
vacunas_por_fabricante.to_csv("vacunas_por_fabricante.csv", index=False)

In [6]:
vac_per_manu_factor_sp = vac_per_manu_factor[vac_per_manu_factor.location == "Spain"]
# Vamos a quedarnos con el mes de enero
vac_per_manu_factor_sp_jan = vac_per_manu_factor_sp[vac_per_manu_factor_sp.date.str.contains("2021-01")]


In [7]:
# Vamos a añadir filas de forma que no tengamos saltos de fechas
# Primero vamos a crear un dataframe con todas las fechas
dates = pd.DataFrame(pd.date_range(start='2021-01-01', end='2021-01-31'), columns=['date'])
dates['date'] = dates['date'].dt.strftime('%Y-%m-%d')
# Ahora vamos a unirlo con el dataframe de vacunas
vac_per_manu_factor_sp_jan_all = pd.merge(dates, vac_per_manu_factor_sp_jan, on='date', how='left')
# Y ahora vamos a rellenar los valores nulos con 0
vac_per_manu_factor_sp_jan_all = vac_per_manu_factor_sp_jan_all.fillna(0)

In [8]:
# Vamos a añadir filas de forma que no tengamos saltos de fechas
# Pero las vamos a duplicar 4 veces
# De forma que el 2021-01-01 se repita 4 veces
dates = pd.DataFrame(pd.date_range(start='2021-01-01', end='2021-12-31'), columns=['date'])
dates['date'] = dates['date'].dt.strftime('%Y-%m-%d')
# Ahora repetimos 4 veces las fechas por el número de paises por el número de fabricantes
dates = pd.concat([dates]* (len(vac_per_manu_factor_sp_jan_all.vaccine.unique()) -1))
# Ordenamos por fecha
dates = dates.sort_values(by=['date'])
#vac_per_manu_factor_sp_jan_all = pd.merge(dates,vac_per_manu_factor_sp_jan, on='date', how='inner')
vacunas = vac_per_manu_factor_sp_jan_all.vaccine.unique()
print(vacunas)
# Elimniamos el primer elemento que es 0
vacunas = vacunas[1:]
# repetimos 4 veces las vacunas
vacunas = pd.concat([pd.Series(vacunas)]*365).tolist()
print(vacunas)
# Ahora vamos a sacar una lista con todos los paises
countries = vac_per_manu_factor_sp_jan_all.location.unique()
# Añadimos las columnas de location vaccine total_vaccinations al dataframe
dates['location'] = "Spain"
# En vaccine vamos a repetir 4 veces el nombre de la vacuna
dates['vaccine'] = vacunas
dates['total_vaccinations'] = 0
# Resetamos el indice
dates = dates.reset_index(drop=True)


[0 'Johnson&Johnson' 'Moderna' 'Oxford/AstraZeneca' 'Pfizer/BioNTech']
['Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'Johnson&Johnson', 'Moderna', 'Oxford/AstraZeneca', 'Pfizer/BioNTech', 'John

In [9]:
dates.head(50)

,date,location,vaccine,total_vaccinations
0,2021-01-01,Spain,Johnson&Johnson,0
1,2021-01-01,Spain,Moderna,0
2,2021-01-01,Spain,Oxford/AstraZeneca,0
3,2021-01-01,Spain,Pfizer/BioNTech,0
4,2021-01-02,Spain,Johnson&Johnson,0
5,2021-01-02,Spain,Moderna,0
6,2021-01-02,Spain,Oxford/AstraZeneca,0
7,2021-01-02,Spain,Pfizer/BioNTech,0
8,2021-01-03,Spain,Johnson&Johnson,0
9,2021-01-03,Spain,Moderna,0


In [10]:
vac_per_manu_factor_sp_jan

,location,date,vaccine,total_vaccinations
42391,Spain,2021-01-08,Johnson&Johnson,0
42392,Spain,2021-01-08,Moderna,0
42393,Spain,2021-01-08,Oxford/AstraZeneca,0
42394,Spain,2021-01-08,Pfizer/BioNTech,420080
42395,Spain,2021-01-15,Moderna,7270
42396,Spain,2021-01-15,Pfizer/BioNTech,921812
42397,Spain,2021-01-22,Moderna,25724
42398,Spain,2021-01-22,Pfizer/BioNTech,1244893
42399,Spain,2021-01-29,Moderna,29626
42400,Spain,2021-01-29,Pfizer/BioNTech,1617030


In [11]:
# Vamos a recorrer el dataframe que contiene el número de vacunas por fabricante
# y vamos a ir completandolo en el dataframe de dates
data = vac_per_manu_factor_sp.reset_index(drop=True)
vacunas = vac_per_manu_factor_sp_jan.vaccine.unique()
vacunas_dict = {}
for vacuna in vacunas:
    vacunas_dict[vacuna] = data[data.vaccine == vacuna].reset_index(drop=True)
print(vacunas_dict)

{'Johnson&Johnson':    location        date          vaccine  total_vaccinations
0     Spain  2021-01-08  Johnson&Johnson                   0
1     Spain  2021-04-23  Johnson&Johnson               34010
2     Spain  2021-04-30  Johnson&Johnson              132301
3     Spain  2021-05-07  Johnson&Johnson              194375
4     Spain  2021-05-14  Johnson&Johnson              228905
..      ...         ...              ...                 ...
57    Spain  2022-10-21  Johnson&Johnson             1981768
58    Spain  2022-11-11  Johnson&Johnson             1981769
59    Spain  2022-12-09  Johnson&Johnson             1981770
60    Spain  2022-12-23  Johnson&Johnson             1981771
61    Spain  2023-01-20  Johnson&Johnson             1981772

[62 rows x 4 columns], 'Moderna':     location        date  vaccine  total_vaccinations
0      Spain  2021-01-08  Moderna                   0
1      Spain  2021-01-15  Moderna                7270
2      Spain  2021-01-22  Moderna               257

In [12]:
# Recorremos el diccionario
for vacuna in vacunas_dict:
    LDWD = dates[dates.vaccine == vacuna].iloc[0].date
    cum_vaccinations = 0
    for i in range(len(vacunas_dict[vacuna])):
        # COgemos la primera fila
        fila = vacunas_dict[vacuna].iloc[i]
        current_date = fila.date
        cum_vaccinations = fila.total_vaccinations - cum_vaccinations
        if i == 0:
            diff_dates = (pd.to_datetime(current_date) - pd.to_datetime(LDWD)).days + 1
        else:
            diff_dates = (pd.to_datetime(current_date) - pd.to_datetime(LDWD)).days
        #print(diff_dates)
        total_vaccinations = cum_vaccinations // diff_dates
        #print(fila.total_vaccinations)
        #print(current_date)
        #print(total_vaccinations)
        cum_vaccinations = fila.total_vaccinations
        dates.loc[(dates.vaccine == vacuna) & (dates.date <= current_date) & (dates.date >= LDWD), 'total_vaccinations'] = total_vaccinations
        LDWD = current_date
        
        

In [13]:
vac_per_manu_factor_sp_jan

,location,date,vaccine,total_vaccinations
42391,Spain,2021-01-08,Johnson&Johnson,0
42392,Spain,2021-01-08,Moderna,0
42393,Spain,2021-01-08,Oxford/AstraZeneca,0
42394,Spain,2021-01-08,Pfizer/BioNTech,420080
42395,Spain,2021-01-15,Moderna,7270
42396,Spain,2021-01-15,Pfizer/BioNTech,921812
42397,Spain,2021-01-22,Moderna,25724
42398,Spain,2021-01-22,Pfizer/BioNTech,1244893
42399,Spain,2021-01-29,Moderna,29626
42400,Spain,2021-01-29,Pfizer/BioNTech,1617030


In [14]:
# Store the dataframe in a csv file
pd.DataFrame(dates).to_csv("vacunas_por_fabricante.csv", index=False)